In [1]:
import pandas as pd
import numpy as np
import math
import json

In [2]:
data, item_to_genre, genres = pd.read_csv("~/Data/anime.csv")[["anime_id", "genre"]].fillna(""), {}, []
for st in data.values:
    item_to_genre[st[0]] = [a.strip() for a in st[1].split(",") if len(a) > 0]
    genres += st[1].split(",")

num_genres = len(set(genres))

In [3]:
user_item = pd.read_csv("~/Data/clean_rating3.csv")
from collections import defaultdict
grouped, corpus, genre_to_user = user_item.groupby("user_id"), {}, defaultdict(list)
for gp in grouped.groups:
    items, sentences = [t[1] for t in grouped.get_group(gp).values if t[2] > 5], []
    for item in items:
        sentences += item_to_genre.get(item, [])
    
    for genre in set(sentences):
        genre_to_user[genre].append(gp)
        
    corpus[gp] = sentences

In [12]:
word_frequency, doc_frequency, word_tf, word_idf = defaultdict(int), defaultdict(int), defaultdict(float), defaultdict(float)
for st in corpus.values():
    for word in st:
        word_frequency[word] += 1
    for word in set(st):
        doc_frequency[word] += 1
        
num_words = sum(word_frequency.values())
for word in word_frequency:
    word_tf[word] = word_frequency[word] / num_words
    
for doc in doc_frequency:
    word_idf[doc] = math.log(len(corpus) / (doc_frequency[doc] + 1))
    
tf_idf = {}
for word in word_tf:
    tf_idf[word] = word_tf[word] * word_idf[word]

sorted_tf_idf = sorted(tf_idf.items(), key = lambda x: x[1], reverse = True)
dic = {'genre': [i[0] for i in sorted_tf_idf], 'tf_idf': [i[1] for i in sorted_tf_idf]}
df = pd.DataFrame(dic)
df.to_csv("tf_idf.csv")

In [6]:
from collections import Counter, defaultdict
genres, tfidfVector = tf_idf.keys(), defaultdict(list)
for user in corpus:
    counter, num = Counter(corpus[user]), len(corpus[user])
    for genre in genres:
        tfidfVector[user].append((counter.get(genre, 0) / num) * tf_idf[genre])

In [29]:
input, candidate, all_user = ["yes", "no", "yes", "yes", "no"], set(corpus.keys()), set(corpus.keys())
liked_genres = []
for idx, ans in enumerate(input):
    if ans == "yes":
        candidate &= set(genre_to_user[sorted_tf_idf[idx][0]])
        liked_genres.append(sorted_tf_idf[idx][0])
    else:
        candidate &= all_user - set(genre_to_user[sorted_tf_idf[idx][0]])

from collections import defaultdict
recommand = defaultdict(int)
for user in candidate:
    animes = user_item[user_item.user_id == user].anime_id.tolist()
    for item in animes:
        overlap = set(item_to_genre[item]) & set(liked_genres)
        if overlap:
            recommand[item] += (int(user_item[(user_item.user_id == user) & (user_item.anime_id == item)].rating.values) - 5) * (len(overlap) / len(liked_genres))

            
print(sorted(recommand.items(), key = lambda x: x[1], reverse = True))

[(3092, 36.99999999999999), (4814, 36.99999999999999), (9926, 36.999999999999986), (741, 31.333333333333332), (11123, 29.666666666666668), (8634, 25.999999999999996), (243, 24.333333333333336), (8063, 23.666666666666668), (149, 22.999999999999996), (21105, 17.66666666666667), (25649, 17.333333333333336), (747, 17.333333333333332), (507, 16.333333333333332), (3210, 16.333333333333332), (5774, 16.333333333333332), (3375, 15.333333333333334), (2238, 14.666666666666666), (11135, 13.666666666666664), (7058, 12.666666666666668), (15933, 12.666666666666663), (719, 12.0), (3656, 12.0), (5902, 12.0), (1532, 11.999999999999998), (31680, 11.666666666666666), (13767, 11.666666666666666), (553, 10.666666666666666), (8449, 10.666666666666666), (20371, 10.666666666666666), (20815, 10.666666666666666), (736, 10.0), (15439, 9.666666666666668), (850, 9.333333333333334), (6553, 9.333333333333334), (4021, 8.666666666666666), (12695, 8.333333333333332), (30346, 8.333333333333332), (5719, 7.999999999999998)

In [11]:
a = {"1": 1}
print()

AttributeError: 'dict_values' object has no attribute 'value'